# Imports

In [1]:
import pandas as pd
import numpy as np
import os

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from numpy.random import normal, uniform
import random

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_validate, KFold, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.neural_network import MLPRegressor

import dill

In [2]:
%matplotlib inline

In [ ]:
# dill.dump_session('NN.db')

In [ ]:
# dill.load_session('NN.db')

In [3]:
SEED = 73 # random seed

# Data reading

In [ ]:
# y_train = pd.read_csv('data/prepared/y_train.csv')
# y_test = pd.read_csv('data/prepared/y_test.csv')

# X_train_standartized = pd.read_csv('data/prepared/X_train_std.csv')
# X_test_standartized = pd.read_csv('data/prepared/X_test_std.csv')

# current_target = y_train.columns[0]

# y_train = y_train.values.reshape(len(y_train))
# y_test = y_test.values.reshape(len(y_test))

In [4]:
data = pd.read_csv('data/prepared/data_processed.csv')

In [5]:
names = data.columns
names

Index(['AP1', 'FD', 'FTa1', 'FTa2', 'FTa3', 'FTb', 'FTc', 'LFY', 'TFL1a',
       'TFL1c'],
      dtype='object')

In [6]:
current_target = names[0]
current_target

'AP1'

In [11]:
# Choose target and standartize the data

y = data[current_target]
X = data.drop(current_target, axis=1)

In [13]:
st_scalers = {}

In [14]:
st_scalers[current_target] = StandardScaler()

In [15]:
X_st = st_scalers[current_target].fit_transform(X)

In [16]:
# train-test split

X_st_train, X_st_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.25, random_state=SEED, shuffle=True)

# NN model

## Model

len(hidden_layer_sizes) = n_layers

* Стоит делать Gread search по $\alpha$, momentum, возможно, начальный learning_rate_init
* Попробовать другие активационные функции
* другая архитектура


In [22]:
model = MLPRegressor(hidden_layer_sizes=(9, 4), solver='sgd', validation_fraction=0.25,
                     n_iter_no_change=50, max_iter=5000, early_stopping=True,
                     learning_rate='adaptive', alpha=1e-4, activation='logistic')

## Rough tuning

In [23]:
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

grid = {
#     'hidden_layer_sizes': [(5, 5), (10, 10), (20, 20)],
    'hidden_layer_sizes': [(10, 10), (5, 5), (10, 5), (5, 10), (20, 10)],
    'alpha': np.geomspace(0.00001, 0.01, 4),
    'activation' : ['logistic', 'relu']
    #'learning_rate_init': np.geomspace(0.001, 0.1, 3)
}

gs = GridSearchCV(model, grid, 
                  n_jobs=-1, 
                  scoring=['neg_mean_squared_error', 'r2'], 
                  refit='neg_mean_squared_error', 
                  cv=cv, 
                  verbose=10)

In [24]:
%%time
gs.fit(X_st_train, y_train);

Fitting 5 folds for each of 40 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:   26.1s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   30.5s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:   36.5s


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [25]:
gs.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [ ]:
gs.best_params_

## Fine tuning

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

grid = {
    'alpha': np.linspace(1e-5, 1e-2, 20).round(5)
}

gs = GridSearchCV(model, grid, 
                  n_jobs=-1, 
                  scoring=['neg_mean_squared_error', 'r2'], 
                  refit='neg_mean_squared_error', 
                  cv=cv, 
                  verbose=10)

In [ ]:
%%time
gs.fit(X_train_standartized, y_train);

In [ ]:
gs.best_params_

In [ ]:
model = gs.best_estimator_

## Prediction

In [ ]:
train_pred = model.predict(X_train_standartized)

In [ ]:
plt.scatter(y_train, train_pred)
plt.plot([min(y_train), max(y_train)], [min(y_train), max(y_train)], 'r')
plt.xlabel('True Values' + ' (' + current_target + ')')
plt.ylabel('Predictions' + ' (' + current_target + ')')
plt.axis('equal')
plt.axis('square')
plt.title('Predictions on train')
plt.xlim(min(y_train) -10, max(y_train)+10)
plt.ylim(min(y_train) -10, max(y_train)+10)
plt.show()
# plt.xlim([0,3])
# plt.ylim([0,3])

In [ ]:
np.corrcoef(train_pred, y_train)

In [ ]:
mean_squared_error(y_train, train_pred)

In [ ]:
predictions = model.predict(X_test_standartized)

In [ ]:
plt.scatter(y_test, predictions)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
plt.xlabel('True Values' + ' (' + current_target + ')')
plt.ylabel('Predictions' + ' (' + current_target + ')')
plt.title('Predictions on test')
plt.axis('equal')
plt.axis('square')
plt.show()
# plt.xlim([0,3])
# plt.ylim([0,3])

In [ ]:
mean_squared_error(y_test, predictions)

In [ ]:
np.corrcoef(predictions, y_test)

In [ ]:
pd.DataFrame(gs.cv_results_).T

In [ ]:
model.loss_

Коэффициент детерминации $R^2$:

In [ ]:
model.score(X_test_standartized, y_test)

In [ ]:
model.coefs_

## Prediction on real

In [ ]:
true_lvls_SD = pd.DataFrame(data['AP1_SD'].days.values, columns = ['days'])
true_lvls_LD = pd.DataFrame(data['AP1_LD'].days.values, columns = ['days'])

for k in data.keys():
    if k.endswith('LD'):
        true_lvls_LD[k] = data[k].rltv_transcription_lvl.values
    else:
        true_lvls_SD[k] = data[k].rltv_transcription_lvl.values

In [ ]:
true_lvls_LD.head()

In [ ]:
true_lvls_SD = true_lvls_SD.rename(columns=lambda x: x[:-3])

true_lvls_LD = true_lvls_LD.rename(columns=lambda x: x[:-3])

In [ ]:
true_lvls = true_lvls_LD.append(true_lvls_SD, ignore_index=True)

true_lvls = true_lvls.drop('d', axis =1)

true_lvls = true_lvls.reindex(sorted(true_lvls.columns), axis=1)

In [ ]:
true_AP1_pred = model.predict(StandardScaler().fit_transform(true_lvls.drop(current_target, axis=1)))

In [ ]:
len(true_lvls['AP1'])

In [ ]:
plt.scatter(true_lvls['AP1'], true_AP1_pred)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
plt.xlabel('True Values' + ' (' + current_target + ')')
plt.ylabel('Predictions' + ' (' + current_target + ')')
plt.axis('equal')
plt.axis('square')
plt.show()
# plt.xlim([0,3])
# plt.ylim([0,3])

In [ ]:
plt.plot(list(range(len(true_AP1_pred))), true_AP1_pred, '-')
plt.plot(list(range(len(true_AP1_pred))), true_lvls['AP1'], 'go')

## LD & SD separate

### LD

In [ ]:
true_AP1_LD_pred = model.predict(st_scaler.fit_transform(true_lvls_LD.drop([current_target, 'd'], axis=1)))

In [ ]:
plt.scatter(true_lvls_LD['AP1'], true_AP1_LD_pred)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r')
plt.xlabel('True Values' + ' (' + current_target + ')')
plt.ylabel('Predictions' + ' (' + current_target + ')')
plt.axis('equal')
plt.axis('square')
plt.show()
# plt.xlim([0,3])
# plt.ylim([0,3])

In [ ]:
np.corrcoef(true_lvls_LD['AP1'], true_AP1_LD_pred)

In [ ]:
mean_squared_error(true_lvls_LD['AP1'], true_AP1_LD_pred)

In [ ]:
plt.plot(true_lvls_LD['d'], true_AP1_LD_pred, '-')
plt.plot(true_lvls_LD['d'], true_lvls_LD['AP1'], 'go')
plt.xlabel('Days after sowing')
plt.ylabel('AP1')
plt.legend(['AP1 prediction', 'True AP1 level'])
plt.title('AP1 expression level for LD conditions')

### SD

In [ ]:
true_AP1_SD_pred = model.predict(st_scaler.transform(true_lvls_SD.drop([current_target, 'd'], axis=1)))

In [ ]:
plt.scatter(true_lvls_SD['AP1'], true_AP1_SD_pred)
plt.plot([min(true_lvls_SD['AP1']), max(true_lvls_SD['AP1'])], 
         [min(true_lvls_SD['AP1']), max(true_lvls_SD['AP1'])], 'r')
plt.xlabel('True Values' + ' (' + current_target + ')')
plt.ylabel('Predictions' + ' (' + current_target + ')')
plt.axis('equal')
plt.axis('square')
plt.show()
# plt.xlim([0,3])
# plt.ylim([0,3])

In [ ]:
np.corrcoef(true_lvls_SD['AP1'], true_AP1_SD_pred)

In [ ]:
mean_squared_error(true_lvls_SD['AP1'], true_AP1_SD_pred)

In [ ]:
plt.plot(true_lvls_SD['d'], true_AP1_SD_pred, '-')
plt.plot(true_lvls_SD['d'], true_lvls_SD['AP1'], 'go')
plt.xlabel('Days after sowing')
plt.ylabel('AP1')
plt.legend(['AP1 prediction', 'True AP1 level'])
plt.title('AP1 expression level for SD conditions')

# Gene interaction analysis

**Mutants**

In [ ]:
true_lvls

In [ ]:
true_lvls.shape

In [ ]:
true_medians = true_lvls.describe().loc['50%']

In [ ]:
mutants = pd.DataFrame(true_lvls, copy=True)
mutants = mutants.drop('AP1', axis=1)

In [ ]:
true_medians

In [ ]:
mutants_out = pd.DataFrame(mutants, copy=True)

for k in mutants_out.keys():
    mutants_out[k] = 0

In [ ]:
for k in mutants.keys():
    mutants[k] = 0
    mutants_out[k] = model.predict(st_scaler.transform(mutants))
    mutants[k] = true_lvls[k]

In [ ]:
mutants_out

In [ ]:
mutants_out_LD = mutants_out.iloc[:14, :]
mutants_out_SD = mutants_out.iloc[14:, :]

In [ ]:
plt.figure(figsize=(20, 17))
plt.title('LD conditions')
for i in range(9):
    gene = X_test_standartized.keys()[i]
    plt.subplot(3, 3, i+1)
    plt.plot(true_lvls_LD['d'], mutants_out_LD[gene])
    plt.plot(true_lvls_LD['d'], true_lvls_LD['AP1'], 'go')
    plt.xlabel(gene + ' knockout')
    plt.ylabel('AP1')
    
    if i == 0:
        plt.legend(['Predictions for mutants', 'True AP1 level'])

plt.show()

In [ ]:
plt.figure(figsize=(20, 17))
plt.title('LD conditions')
for i in range(9):
    gene = X_test_standartized.keys()[i]
    plt.subplot(3, 3, i+1)
    plt.plot(true_lvls_SD['d'], mutants_out_SD[gene])
    plt.plot(true_lvls_SD['d'], true_lvls_SD['AP1'], 'go')
    plt.xlabel(gene + ' knockout')
    plt.ylabel('AP1')
    
    if i == 0:
        plt.legend(['Predictions for mutants', 'True AP1 level'])

plt.show()

In [ ]:
integrate_level_AP1_LD = np.trapz(true_lvls_LD['AP1'])
integrate_level_AP1_SD = np.trapz(true_lvls_SD['AP1'])

integrate_levels_mutant_LD = {}
integrate_levels_mutant_SD = {}

for k in mutants_out_SD.keys():
    integrate_levels_mutant_SD[k] = np.trapz(mutants_out_SD[k]).round(2)
    integrate_levels_mutant_LD[k] = np.trapz(mutants_out_LD[k]).round(2)

In [ ]:
for key, value in sorted(integrate_levels_mutant_SD.items(), key=lambda item: item[1], reverse = True):
    print("%s:\t %s" % (key, value.round(3)))

print('True level', integrate_level_AP1_SD)

In [ ]:
for key, value in sorted(integrate_levels_mutant_LD.items(), key=lambda item: item[1], reverse = True):
    print("%s:\t %s" % (key, value.round(3)))
    
print('True level', integrate_level_AP1_LD)